In [1]:
from sympy import *
import math
import numpy as np

### 11.2

In [2]:
h = Symbol('h')
A = Symbol('A')
ф_f = Symbol('ф_f')

In [3]:
km = Matrix([[h*A, 0],
           [0,0]])

In [4]:
km

Matrix([
[A*h, 0],
[  0, 0]])

In [5]:
fs = Matrix([h*A*ф_f,0])

In [6]:
fs

Matrix([
[A*h*ф_f],
[      0]])

### 11.3

In [7]:
def FinAssemble(K,k,i,j):
    K[i,i] = K[i,i] + k[0,0]
    K[i,j] = K[i,j] + k[0,1]
    K[j,i] = K[j,i] + k[1,0]
    K[j,j] = K[j,j] + k[1,1]
    return K

In [8]:
def ElementForceQ(h,P,L,ф_f):
    return h*P*L*ф_f/2*np.array([1,1])

In [9]:
def ElementForceS(h,P,L,ф_f,A):
    return h*P*L*ф_f/2*np.array([1,1])+h*A*ф_f*np.array([0,1])

In [10]:
def ForceAssemble(F,f,i,j):
    F[i]=F[i]+f[0]
    F[j]=F[j]+f[1]
    return F

In [11]:
def ElementStiffness(k,A,L,P,h):
    return k*A/L*np.matrix([[1,-1], [-1,1]]) + h*P*L/6*np.matrix([[2,1], [1,2]])

In [12]:
def ElementStiffnessM(kc,A,L,P,h):
    return (kc*A/L*np.matrix([[1,-1], [-1,1]]) 
            + h*P*L/6*np.matrix([[2,1], [1,2]])
            +np.matrix([[0,0], [0,h*A]]))

In [13]:
def FEA_ф(inp, bc_node, bc_val):
    K=np.zeros(shape=(len(inp)+1, len(inp)+1))
    
    F=np.zeros(shape=(len(inp)+1,))
    
    max_coord=0
    for t, el in enumerate(inp):
        Xi, Xj, kc, A, P, h, ф_f, i, j = el
        max_coord=max(max_coord,i,j)
        if t==len(inp)-1:
            k=ElementStiffnessM(kc,A,Xj-Xi,P,h)
            f=ElementForceS(h,P,Xj-Xi,ф_f,A)
        else:
            k=ElementStiffness(kc,A,Xj-Xi,P,h)
            f=ElementForceQ(h,P,Xj-Xi,ф_f)
        K=FinAssemble(K, k, i, j)
        F=ForceAssemble(F,f,i,j)
        
#     https://github.com/CALFEM/calfem-matlab/blob/master/fem/solveq.m
#     https://www.byggmek.lth.se/fileadmin/byggnadsmekanik/publications/tvsm5000/web5167.pdf p.35  

#     F = np.zeros(shape =(6, 1)) 
#     F[2] = 10
     
     
#     K=np.matrix([[ 25. , -25. , 0. , 0. , 0. , 0. ],
#         [-25. , 49.3, -24.3, 0. , 0. , 0. ],
#         [ 0. , -24.3, 24.7, -0.4, 0. , 0. ],
#         [ 0. , 0. , -0.4, 17.4, -17. , 0. ],
#         [ 0. , 0. , 0. , -17. , 24.7, -7.7],
#         [ 0. , 0. , 0. , 0. , -7.7, 7.7]])
    
#     bc = np.array([[0, 5],
#            [-17.0, 20.0]]).astype(int).T
    
    bc=np.array([bc_node, 
                bc_val]).T

    nd, nd=K.shape
    fdof=np.array([i for i in range(nd)]).T
    d=np.zeros(shape=(len(fdof),))
    Q=np.zeros(shape=(len(fdof),))


    pdof=bc[:,0].astype(int)
    dp=bc[:,1]
    fdof=np.delete(fdof, pdof, 0)
    s=np.linalg.solve(K[fdof,:][:,fdof],(F[fdof].T-np.dot(K[fdof,:][:,pdof],dp.T)).T)
    d[pdof]=dp
    d[fdof]=s.reshape(-1,)
    Q=np.dot(K,d).T-F
    
    return d, Q

In [14]:
kc=3
A=4
P=10
h=0.1
ф_f=20

In [15]:
inp = [[0,1,kc,A,P,h,ф_f,0,1],
       [1,3,kc,A,P,h,ф_f,1,2],
       [3,5,kc,A,P,h,ф_f,2,3],
       [5,8,kc,A,P,h,ф_f,3,4]]

In [16]:
d,Q = FEA_ф(inp, [0],[80.])

In [17]:
d

array([80.        , 65.04539384, 45.74043816, 35.52034301, 30.05948158])

In [18]:
Q

array([ 2.06962840e+02,  0.00000000e+00, -1.13686838e-13,  7.10542736e-14,
       -1.42108547e-14])

In [19]:
inp = [[0,2,kc,A,P,h,ф_f,0,1],
       [2,4,kc,A,P,h,ф_f,1,2],
       [4,6,kc,A,P,h,ф_f,2,3],
       [6,8,kc,A,P,h,ф_f,3,4]]

In [20]:
d,Q = FEA_ф(inp, [0],[80.])

In [21]:
d

array([80.        , 53.94556119, 39.87190868, 32.811871  , 30.27367014])

In [22]:
### 11.5

In [23]:
### 11.8

In [24]:
### 11.9

In [25]:
### 11.11